In [1]:
%run ./../../utils/commonImports.py
%matplotlib inline

# Load Data

In [2]:
db = 'D:\\Dropbox\\My work\\krypl-project\\sqlite\\ploniex-chart-data\\USDT_BTC.db'
data = load_trading_data(db, 'chart_data', from_date='2016-01-01', period='5min')

In [3]:
data.head()

,timestamp,period,open,high,low,close,volume,quoteVolume,weightedAverage,date
0,1451606400,5min,430.5732,430.5732,430.5732,430.5732,0.0000,0.0000,430.5732,2016-01-01 00:00:00
1,1451606700,5min,428.2600,428.2600,428.2600,428.2600,76.5618,0.1788,428.2600,2016-01-01 00:05:00
2,1451607000,5min,428.2600,428.2600,428.2600,428.2600,0.0000,0.0000,428.2600,2016-01-01 00:10:00
3,1451607300,5min,428.2600,428.2600,428.2600,428.2600,0.0000,0.0000,428.2600,2016-01-01 00:15:00
4,1451607600,5min,428.2600,428.2600,428.2600,428.2600,0.0000,0.0000,428.2600,2016-01-01 00:20:00


# Test Data - Timestamp diff

In [4]:
diffs = data['timestamp'].shift(-1) - data['timestamp']
data['timestampDiff'] = diffs

In [5]:
diffs.value_counts()

300.0000    290463
Name: timestamp, dtype: int64

In [6]:
zero_diff = data.query('timestampDiff == 1200')
zero_diff

,timestamp,period,open,high,low,close,volume,quoteVolume,weightedAverage,date,timestampDiff


# Day Difference Calc

In [7]:
from utils.date import timestamp_to_date
from datetime import timedelta
import time
   

ohlc = data[['timestamp', 'period', 'open', 'high', 'low', 'close', 'date']].copy()
ohlc['day'] = ohlc['timestamp'].apply(timestamp_to_date).apply(lambda x: x.date())
ohlc['yesterday'] = ohlc['day'] - timedelta(days=1)

grouped = ohlc.groupby('day').agg(['min', 'max', 'first', 'last'])
_open = grouped['open']['first']
high = grouped['high']['max']
low = grouped['low']['min']
close = grouped['close']['last']
mean_candle = ((_open + high + low + close) / 4) \
    .to_frame('yesterdayMeanCandle')\
    .reset_index()\
    .rename(columns={'day': 'yesterday'})

ohlc = ohlc.merge(mean_candle, how='inner', on='yesterday')

In [9]:
data.shape

(290464, 11)

In [8]:
ohlc.shape

(290176, 10)

In [13]:
df = pd.DataFrame({'a': [0, 1], 'b': [3, 4]})
a, b, c = df.iloc[0, :]
a, b

ValueError: not enough values to unpack (expected 3, got 2)

In [17]:
def day_group(col):
    return ohlc.groupby('day')[col].apply(list).to_frame(col)

def mean_candle_group(row):
    opens, highs, lows, closes, dates = row
    mean_candles = []
    for i in range(len(row['open'])):
        mean_candle = np.mean([opens[0], max(highs[:i+1]), min(lows[:i+1]), closes[i]])
        mean_candles.append(mean_candle)
    
    return dates, mean_candles

dates = day_group('date')
opens = day_group('open')
highs = day_group('high')
lows = day_group('low')
closes = day_group('close')

day_groups = opens.join(highs).join(lows).join(closes).join(dates)
mean_candle_groups = day_groups.apply(mean_candle_group, axis=1).to_frame('date_candles_tuple').reset_index()

In [18]:
mean_candle_groups.head()

,day,date_candles_tuple
0,2016-01-02,"([2016-01-02 00:00:00, 2016-01-02 00:05:00, 20..."
1,2016-01-03,"([2016-01-03 00:00:00, 2016-01-03 00:05:00, 20..."
2,2016-01-04,"([2016-01-04 00:00:00, 2016-01-04 00:05:00, 20..."
3,2016-01-05,"([2016-01-05 00:00:00, 2016-01-05 00:05:00, 20..."
4,2016-01-06,"([2016-01-06 00:00:00, 2016-01-06 00:05:00, 20..."


In [20]:
dates, mean_candles = [], []
for i in range(mean_candle_groups.shape[0]):
    dates += mean_candle_groups.loc[i, 'date_candles_tuple'][0]
    mean_candles += mean_candle_groups.loc[i, 'date_candles_tuple'][1]

today_mean_candles = pd.DataFrame({'todayMeanCandle': mean_candles, 'date': dates})
ohlc = ohlc.merge(today_mean_candles, how='inner', on='date')

In [21]:
ohlc['dayDifference'] = (ohlc['todayMeanCandle'] - ohlc['yesterdayMeanCandle']) / ohlc['yesterdayMeanCandle'] 

In [24]:
ohlc.head()

,timestamp,period,open,high,low,close,date,day,yesterday,yesterdayMeanCandle,todayMeanCandle,dayDifference
0,1451692800,5min,434.9900,434.9900,434.9900,434.9900,2016-01-02 00:00:00,2016-01-02,2016-01-01,432.2058,434.9900,0.0064
1,1451693100,5min,434.9900,435.0000,434.9900,435.0000,2016-01-02 00:05:00,2016-01-02,2016-01-01,432.2058,434.9950,0.0065
2,1451693400,5min,435.0000,435.0000,435.0000,435.0000,2016-01-02 00:10:00,2016-01-02,2016-01-01,432.2058,434.9950,0.0065
3,1451693700,5min,435.0000,435.0000,435.0000,435.0000,2016-01-02 00:15:00,2016-01-02,2016-01-01,432.2058,434.9950,0.0065
4,1451694000,5min,435.0000,435.0000,435.0000,435.0000,2016-01-02 00:20:00,2016-01-02,2016-01-01,432.2058,434.9950,0.0065


In [25]:
write_tsv(ohlc, 'D:\\Dropbox\\My work\\krypl-project\\dataFeatures\\day-difference.tsv')